In [ ]:
!pip install torch


In [ ]:
!pip install transformers peft huggingface_hub

In [ ]:
!pip install peft deepspeed optimum accelerate

In [ ]:
!pip install -U bitsandbytes

In [ ]:
import torch
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print("No GPU available")

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from huggingface_hub import login

login(token="")

/home/karthick/finetuning/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/karthick/.cache/huggingface/token
Login successful


In [ ]:
base_model_name = "Qwen/Qwen2-1.5B"
adapter_model_name = "tryhighlight/qwen2-1.5B-task-detection"
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
)

In [ ]:
base_model_name = "microsoft/Phi-3-mini-4k-instruct"
adapter_model_name = "tryhighlight/phi3-mini-task-dataset"
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="auto",  
    trust_remote_code=True,
)

In [6]:
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
adapter_model_name = "/home/karthick/Qwen2/examples/sft/llama-3-8b-task-detect/checkpoint-9000"
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    attn_implementation = "eager",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
)

Loading checkpoint shards: 100%|███████████████████████████████████████████| 4/4 [00:08<00:00,  2.07s/it]


In [7]:
config = PeftConfig.from_pretrained(adapter_model_name)

In [8]:
model = PeftModel.from_pretrained(model, adapter_model_name)

RuntimeError: Error(s) in loading state_dict for PeftModelForCausalLM:
	size mismatch for base_model.model.model.embed_tokens.weight: copying a param with shape torch.Size([128258, 4096]) from checkpoint, the shape in current model is torch.Size([128256, 4096]).
	size mismatch for base_model.model.lm_head.weight: copying a param with shape torch.Size([128258, 4096]) from checkpoint, the shape in current model is torch.Size([128256, 4096]).

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [ ]:
print(tokenizer.special_tokens_map)

In [ ]:
model.eval()  # Set the model to evaluation mode

In [ ]:
# Example usage
system_prompt = "You are a helpful ai assistant. User will provide full name followed by some email or messaging conversation seen on his/her computer screen. Looking at the conversation, detect if there are any TODOs that the above user has to complete as a result of the conversation. If yes, just provide the short single line task that can be directly added to the todo list. If there is no task detected as a TODO, just output the exact phrase \"No task\". If the conversation is about a promotional or advertisement related, please output \"No task\". If the conversation is directed or addressed to someone else, then output \"No task\"."

In [ ]:
ocr_text = "8 hhightignt (Chane -Medal-s' x |) | | | I | | I |\nΓé¼ > G@ 8% appslack.com/client/T037W748A/CO6ADHR1G3F?cdn_fallback=1 * O10:\n& ┬⌐ Search Medal Q @\nMedal v = Gi @highlight ┬⌐ _ bttos:/app.amplitude.com/analytics/medal/chart/new/fmitanwb BBB x 63┬╗ Weamas\n@Atharva what's the latest on the non serverless endpoints so the warm up time is better - and what's up with handling lang context? The APIs shoul \= nati ~┬Ñ this nd users should be informed.\ne ┬⌐ Threads @Josh / @Sam when will screenshots work normally in HL Chat? Do we need to do more designs to unblock team next week? What work do we nee. Mond┬Ñ-July 15th ~ cht chat to a great UX quickly?\nE> Drafts & sent @Maximilian Maksutovic when can we integrate conversations with floaty? What's blocking it from being able to open from attachments? How are we going to fit it into onboarding? These questions all need answers and progress towards\nHome @Mark what's holding us back from more marketing channels? (edited)\na ~ External Connections BD BB 2 eeviies Last realy 3 days ago |\nMs ##-_ medal-akamai @ Pim 74sane\n) _* shated-medat-honeycomb @lulian i had a note from last week that you'd ship the auth skipping - did that happen last week?\npetty ΓÇö-┬Ñ Channels BBD ┬╗ vies Last ents das aso\nOW sees a Pim 10:21am\nQ dt beckend replied to a thread: IMG_1200\nLater @lulian *\n# builds 2 @\n1 Γé¼\n4 chatgpt\nsts 4 community g Sukkrit 6:37 eM\noo <M Hiteam, down with fever, will rest up for the day\n- weed New\nvon fF eacombesuer - Canna:\n# data-and-experiments ┬« Brandon ΓÇ£> 6:26 pm\n3D Tried using Highlight Chat on my windows this weekend and it just responded to me with an empty message. I'l see if I can reproduce it\n4 desktop-frontend\na me\n# desktop-github\nper BB 2 enea\n& highlight a Pim 7.03PM\n# incidents replied to a thread: \"Cmd .\" And now highlight is frozen\nA * @Karthick can we get to the bottom of this?\n┬⌐ incoming-medal-\nΓÇö e\n4 Issue-tracking .\new newer replies\n& leaders\n& Pim 7:10PM\na ED replied to a thread: \"Cmd.:\" And now highlight is frozen\n# move we already are not any new users - the least we can do is make sure our existing users aren't churning due to crahses, gotta prioritize this\n# nyclife\n# random\n+ essage @highlight\n# releases ┬░\nfA #  sentry-alerts tha O@ Ge OB\nLA it ser ┬Ñ More unreads\n"

In [ ]:
import transformers
from datasets import load_dataset
from typing import Dict, List, Optional

max_len = 1024

dataset_name = "tryhighlight/task_dataset"
#Importing the dataset
input_dataset = load_dataset(dataset_name, split="train")
print (input_dataset)

In [ ]:
no_task = 0
for row in input_dataset:
    if row["TASK"] == "No task":
        no_task += 1
print (no_task)

In [ ]:
test = input_dataset.shuffle(seed=65).select(range(100))
for row in test:
    task = row["TASK"]
    print (task)

In [ ]:
# Get the size of the dataset
dataset_size = len(input_dataset)
print(f"Size of the dataset: {dataset_size} samples")
#    dataset = dataset.shuffle(seed=65).select(range(10000)) # Only use 1000 samples for quick demo
def check_length(row):
    # This function checks if the tokenized length is within the max length allowed
    input_content = tokenizer.encode(system_prompt + row["NAME"] + row["CONVERSATION"] + row["TASK"],
                                        add_special_tokens=True,
                                        truncation=False,
                                        return_length=True,
                                        max_length=None)
    return len(input_content) <= max_len - 20 # extra 20 tokens for the chat template

# Filter the dataset to exclude entries that are too long
filtered_dataset = input_dataset.filter(check_length)
print(f"Size of the dataset after filtering: {len(filtered_dataset)} samples")


In [ ]:
def format_chat_template(row):
    row_json = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "My name is " + row["NAME"] + " \n" + row["CONVERSATION"]}]
    tokenized_output = tokenizer.apply_chat_template(
                            row_json,
                            tokenize=True,
                            add_generation_prompt=False,
                            padding=False,
                            max_length=max_len,
                            truncation=True,
                    )
    input_ids = torch.tensor(tokenized_output, dtype=torch.int)

    return dict(
                input_ids=input_ids, user="My name is " + row["NAME"] + " \n" + row["CONVERSATION"]
            )

dataset = filtered_dataset.map(
    format_chat_template,
    num_proc=8,
)
print("Column names in the dataset:", dataset.column_names)


In [ ]:
dataset

In [ ]:
eval_dataset = dataset.shuffle(seed=65).select(range(100))

In [ ]:
eval_dataset

In [ ]:
for row in eval_dataset:
    input_ids = row['input_ids']
    input_ids = torch.tensor(input_ids, dtype=torch.int)
    input_ids = input_ids.to(model.device)
    # Ensure input_ids is 2D
    if input_ids.dim() == 1:
        input_ids = input_ids.unsqueeze(0)
    # Create an attention mask for the input_ids
    attention_mask = torch.ones_like(input_ids, device=model.device)

    outputs = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=50, pad_token_id=tokenizer.pad_token_id, do_sample=True, temperature=1.0)
    outputs[input_ids.shape[1]:]
    text_output = tokenizer.decode(outputs[0][input_ids.shape[1]:], skip_special_tokens=True)
    #print ('input : ', row['user'])
    print ('Expected output : ', row['TASK'])
    cleaned_output = text_output.replace("assistant\n", "")
    print ('Inference output : ', cleaned_output)

In [ ]:
token_text = tokenizer.decode(200)
token_text

In [ ]:
print (input_ids)

In [ ]:
print (attention_mask)

In [ ]:
eos_indices = (input_ids == tokenizer.eos_token_id).nonzero(as_tuple=True)[0]
len(eos_indices)

In [ ]:
eos_indices[0]

In [ ]:
first_eos_index = eos_indices[0]
labels[first_eos_index] = tokenizer.eos_token_id

In [ ]:
labels

In [ ]:
attention_mask[first_eos_index] = 1

In [ ]:
print (attention_mask)

In [ ]:
!python -V